In [3]:
import numpy as np
import pandas as pd
import awkward as ak
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
import matplotlib.pyplot as plt
import topcoffea.modules.utils as utils
import pickle
import gzip
import logging
import time

In [6]:
fSMEFT = "/afs/crc.nd.edu/user/h/hnelson2/dctr/analysis/dctr_SMEFT.pkl.gz"
# fpowheg = "/afs/crc.nd.edu/user/h/hnelson2/dctr/analysis/dctr_powheg.pkl.gz" 

In [7]:
rando=1234

In [8]:
inputs_smeft= pickle.load(gzip.open(fSMEFT)).get()
smeft_nevents = inputs_smeft.shape[0]

In [9]:
inputs_smeft.shape

(1958489, 11)

In [10]:
# inputs_powheg = pickle.load(gzip.open(fpowheg)).get()
inputs_powheg = ((pickle.load(gzip.open(fSMEFT)).get()).sample(frac=1, random_state=rando).reset_index(drop=True)).iloc[:smeft_nevents]

In [11]:
smeft_train = inputs_smeft.sample(frac=0.7, random_state=rando)
smeft_test = inputs_smeft.drop(smeft_train.index)

In [12]:
powheg_train = inputs_powheg.sample(frac=0.7, random_state=rando)
powheg_test = inputs_powheg.drop(powheg_train.index)

In [13]:
truth_smeft = np.ones_like(smeft_train['weights'])
truth_powheg = np.zeros_like(powheg_train['weights'])

weights_smeft = np.ones_like(smeft_train['weights'])
weights_powheg = np.ones_like(powheg_train['weights'])

In [14]:
z = np.concatenate([smeft_train, powheg_train], axis=0).astype(np.float32)
w = np.concatenate([weights_smeft, weights_powheg], axis=0).astype(np.float32)
y = np.concatenate([truth_smeft, truth_powheg], axis=0).astype(np.float32)

In [18]:
# t_test = torch.from_numpy(z)

In [19]:
# train_dataloader = DataLoader(t_test, batch_size=128, shuffle=True)

In [23]:
input_dim = torch.from_numpy(z).shape[1]

In [4]:
class MyDataset(Dataset):
    def __init__(self, data, weights, targets):
        self.data = data
        self.weights = weights
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        weights = self.weights[idx]
        target = self.targets[idx]
        return sample, weights, target

In [15]:
train_dataset = MyDataset(torch.from_numpy(z), torch.from_numpy(w), torch.from_numpy(y))

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [25]:
device = "cpu"
print(f"Using {device} device")
print(f"input dims={input_dim}")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.main_module= nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(),
            nn.Linear(256,256),
            nn.LeakyReLU(),
            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.main_module(x)

model = NeuralNetwork(input_dim).to(device)
# print(model)

Using cpu device
input dims=11


In [26]:
loss_fn = nn.BCELoss(reduction='mean')

In [27]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [41]:
def train(model, dataloader, loss_fn, optimizer, device="cpu"):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0.0
    
    for batch_samples, batch_weights, batch_targets in dataloader:
        batch_samples, batch_weights, batch_targets = (
            batch_samples.to(device),
            batch_weights.to(device),
            batch_targets.to(device)
        )

        # Compute prediction error
        outputs = model(batch_samples).squeeze(1)
        loss = loss_fn(outputs, batch_targets)
        final_loss = torch.mean(loss)

        optimizer.zero_grad()
        final_loss.backward()
        optimizer.step()

        total_loss += final_loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Average Training Loss: {avg_loss:.4f}")

        # # Backpropagation
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

        # # if batch % 100 == 0:
        # loss, current = loss.item(), (batch + 1) * len(X)
        # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [42]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Average Training Loss: 0.7485
Epoch 2
-------------------------------
Average Training Loss: 0.6933
Epoch 3
-------------------------------
Average Training Loss: 0.6933
Epoch 4
-------------------------------
Average Training Loss: 0.6933
Epoch 5
-------------------------------
Average Training Loss: 0.6932
Done!


In [30]:
# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")